In [2]:
import json
import requests
import numpy as np
import pandas as pd

#### Random country choice

In [189]:
data = open('data.json')
countries_list = json.load(data)
country_chosen = 'Finland' #np.random.choice(countries_list)['Name']
country_chosen

'Finland'

#### List of institutions in the country

In [190]:
URL = "https://inspirehep.net/api/institutions?q={}".format(country_chosen)
institution_link = requests.get(URL)

legacy_ICN = []
for institution in institution_link.json().get('hits').get('hits'):
    legacy = institution.get('metadata').get('legacy_ICN')
    number_of_papers =institution.get('metadata').get('number_of_papers')

    if number_of_papers>0:
        legacy_ICN.append(legacy)

legacy_ICN

['Helsinki, Academy of Finland',
 'UEF, Kuopio',
 'Unlisted, FI',
 'VALMET Automation, Tampere',
 'Nevanlinna Inst., Helsinki',
 'IVO Intl., Helsinki',
 'VTT Electronics, Espoo',
 'Espoo, Tech. Research Ctr.',
 'Picker Nordstar, Helsinki',
 'Amateur Astron. Assoc., Seulaset']

#### Getting articles with less than 10 authors for first institution found

In [191]:
literature_link ="https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{}+and+ac+1->+10".format(legacy_ICN[0].replace(' ','+'))
literature_link = requests.get(literature_link)

articles = [article.get('metadata') for article in literature_link.json().get('hits').get('hits')]
len(articles)

10

#### Getting  authors profile from  first article found

In [192]:
articles[0].get('titles')[0]['title'] #name of the article

'Bayesian inference of the fluctuating proton shape in DIS and hadronic collisions'

In [193]:
#extracting authors for first article
authors_profile = [author.get('record').get('$ref') for author in articles[0].get('authors')]
authors_profile

['https://inspirehep.net/api/authors/1074574',
 'https://inspirehep.net/api/authors/1027900',
 'https://inspirehep.net/api/authors/1279546',
 'https://inspirehep.net/api/authors/1616462']

In [196]:
def get_info(URL):
    info = requests.get(URL).json().get('metadata')

    name = info.get('name').get('value')

    email = info.get('email_addresses')[0].get('value') #first valid email adress

    last_position = info.get('positions')[0].get('rank') #last position

    intitution = info.get('positions')[0].get('institution') #institution where they  had their last position

    start = info.get('positions')[0].get('start_date')

    end = info.get('positions')[0].get('end_date')
    
    return(name , email, last_position,intitution,start,end)

In [197]:
_name = []
_email= []
_last_position= []
_intitution= []
_start= []
_end= []
for author in authors_profile:
    name, email, last_position,intitution,start,end = get_info(author)
    _name.append(name )
    _email.append(email)
    _last_position.append(last_position)
    _intitution.append(intitution)
    _start.append(start)
    _end.append(end)

In [204]:
info = {'name ': _name ,
        'email': _email,
        'last_position': _last_position,
        'intitution': _intitution,
        'start': _start,
        'end': _end}


In [205]:
pd.DataFrame(info)

,name,email,last_position,intitution,start,end
0,"Mäntysaari, Heikki",heikki.mantysaari@jyu.fi,JUNIOR,Jyvaskyla U.,2021,None
1,"Schenke, Björn Peter",bschenke@bnl.gov,SENIOR,Brookhaven,2017,None
2,"Shen, Chun",chunshen@wayne.edu,JUNIOR,Wayne State U.,2018,None
3,"Zhao, Wenbin",1501110084@pku.edu.cn,None,Hua-Zhong Normal U.,2020,2021
